In [ ]:
def VRP(Clientes,Volumen,Matriz_de_tiempo,capacidad,TiempoDeCarga,rampas):
    import pandas as pd
    import math
    HorasL = 480 #8 Horas en min
    Rampas = rampas
    Rutas = []
    Rendimientos = []
    end = False
    Numero_camiones = 1
    while end == False:
        T_ClienteMasCercano = 1000000
        ClienteMasCercano = ''
        clientesNovvisitados = []
        Vol_clientesNovvisitados = []
        Ruta = ['Depot']
        Volumen_total = 0
        TiempoTotal = math.ceil((Numero_camiones/Rampas)*TiempoDeCarga)
        for cliente in range(len(Clientes)):
            c = Clientes[cliente]
            if Matriz_de_tiempo[c]['Depot']<T_ClienteMasCercano:
                T_ClienteMasCercano = Matriz_de_tiempo[c]['Depot']
                v = Volumen[cliente]
                ClienteMasCercano = c
        Ruta.append(ClienteMasCercano)
        TiempoTotal+=T_ClienteMasCercano
        Volumen_total+=v
        for noVisitados in range(len(Clientes)):
            if Clientes[noVisitados] != ClienteMasCercano:
                clientesNovvisitados.append(Clientes[noVisitados])
                Vol_clientesNovvisitados.append(Volumen[noVisitados])
        condiciones = False
        while condiciones == False :
            Origen = Ruta[-1]
            Destinos = []
            for i in range(len(clientesNovvisitados)):
                destino = clientesNovvisitados[i]
                tiempo = Matriz_de_tiempo[destino][Origen]
                regreso = Matriz_de_tiempo['Depot'][destino]
                carga = Vol_clientesNovvisitados[i]
                Destinos.append([destino,tiempo,carga,regreso])
            lista_ordenada = sorted(Destinos, key=lambda x: x[1])
            if lista_ordenada:
                DestinoCercano = lista_ordenada[0][0]
                TiempoaDestino = lista_ordenada[0][1]
                CargaDestino = lista_ordenada[0][2]
                TiempoRegreso = lista_ordenada[0][3]
            else:
                condiciones = True
                continue
            if (TiempoTotal+TiempoaDestino+TiempoRegreso)<=HorasL:
                if CargaDestino+Volumen_total<=capacidad:
                    Ruta.append(DestinoCercano)
                    TiempoTotal+=TiempoaDestino
                    Volumen_total+=CargaDestino
                    RestantesC = []
                    RestantesV = []
                    for x in range(len(lista_ordenada)):
                        if lista_ordenada[x][0]!=DestinoCercano:
                            RestantesC.append(lista_ordenada[x][0])
                            RestantesV.append(lista_ordenada[x][2])
                    clientesNovvisitados = RestantesC
                    Vol_clientesNovvisitados = RestantesV
                    condiciones=False
                else:
                    condiciones=True
            else:
                condiciones=True
        ultimoDestino = Ruta[-1]
        TiempoaBodega = Matriz_de_tiempo['Depot'][ultimoDestino]
        TiempoTotal +=TiempoaBodega
        EspacioNoUtilizado = capacidad-Volumen_total
        TiempoSobrante = HorasL-TiempoTotal
        Rendimiento = [Numero_camiones,TiempoTotal,EspacioNoUtilizado,TiempoSobrante]
        Rendimientos.append(Rendimiento)
        Ruta.append('Depot')
        Rutas.append(Ruta)
        if len(clientesNovvisitados)>0:
            Clientes = clientesNovvisitados
            Volumen = Vol_clientesNovvisitados
            Numero_camiones +=1
            end = False
        else:
            end = True
    columnasRendimiento = ['Ruta','Tiempo total de la ruta','Espacio sobrante del camion','Tiempo sobrante de la ruta']
    df_rendimiento = pd.DataFrame(Rendimientos, columns=columnasRendimiento)
    return Rutas,df_rendimiento.to_string(index=False)

In [ ]:
def imprimirRutas(Rutas):
    for i in range(len(Rutas)):
        ruta_formateada = ""
        for j in Rutas[i]:
            ruta_formateada += f"{j} ==> "
        ruta_formateada = ruta_formateada[:-5]
        print(f'Ruta {i+1}')
        print(ruta_formateada)
        print()
        print()

In [ ]:
Clientes=['Cliente_1','Cliente_2','Cliente_3','Cliente_4','Cliente_5','Cliente_6']
Volumen = [20,14,3,40,12,16]
MaT = pd.read_csv("Matriz_de_tiempo.csv",index_col=0)
capacidad = 40
TiempoCarga = 60
rampas = 1
Rutas, Rendimientos = VRP(Clientes,Volumen,MaT,capacidad,TiempoCarga,rampas)

imprimirRutas(Rutas)
print(Rendimientos)

Ruta 1
Depot ==> Cliente_5 ==> Cliente_3 ==> Depot


Ruta 2
Depot ==> Cliente_1 ==> Cliente_6 ==> Depot


Ruta 3
Depot ==> Cliente_4 ==> Depot


Ruta 4
Depot ==> Cliente_2 ==> Depot


 Ruta  Tiempo total de la ruta  Espacio sobrante del camion  Tiempo sobrante de la ruta
    1                       87                           25                         393
    2                      154                            4                         326
    3                      216                            0                         264
    4                      287                           26                         193
